<a href="https://colab.research.google.com/github/PENROG21/PySpark/blob/main/DataFrame_new_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

Схема DataFrame

**2. `StructType` и `StructField`**

* Импортируются из `pyspark.sql.types`.
* `StructType` используется для определения схемы DataFrame, а `StructField` используется для определения отдельных полей в схеме.
* В примере кода `StructType` используется для создания схемы с полями `name` и `age`, каждый из которых имеет тип `StringType` и `IntegerType` соответственно.

**3. `StringType` и `IntegerType`**

* Импортируются из `pyspark.sql.types`.
* `StringType` и `IntegerType` представляют собой встроенные типы данных PySpark, которые используются для определения типов полей в схеме DataFrame.
* В приведенном выше коде поле `name` имеет тип `StringType`, а поле `age` имеет тип `IntegerType`.

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import avg, max


spark = SparkSession.builder \
.appName('schema').config("spark.master", "local").getOrCreate()

Первй способ как создать DataFrame

Этот способ использует схему, котроую вы сами напишите.

In [22]:
# Список кортежей с данными
data = [('Alice', 25),
        ('Artem', 26),
        ("Ivan", 48)]

# Определение схемы DataFrame
# True = это значит поле может быть None
schema = StructType([
    StructField("name", StringType(), True),  # Поле "name" типа String (строка)
    StructField("age", IntegerType(), True),  # Поле "age" типа Integer (целое число)
])

# Создание DataFrame из данных и схемы
df = spark.createDataFrame(data, schema)

# Вывод схемы DataFrame
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)



Вывод данных можно делать через легендарный метод в Spark и мой самый любимый .show

Синтаксис

n - необязательный элемент. Колличество первых строк которые надо вывести.
Максимум строк, который этот метод может вывести это 20.

```
DataFrame.show(n)
```



In [ ]:
df.show()

+-----+---+
| name|age|
+-----+---+
|Alice| 25|
|Artem| 26|
| Ivan| 48|
+-----+---+



Второй способо.

Он не требует создание от вас схемы. Метод createDataFrame сам создаёт схему и определяет тип данных.

In [ ]:
# Список словарей с данными
data = [
    {"name": "Alice", "age": 29},
    {"name": "Bob", "age": 22},
    {"name": "George", "age": 30}
]

# Создание DataFrame из списка словарей
# Spark автоматически выведет схему DataFrame
df = spark.createDataFrame(data)

# Вывод схемы DataFrame
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



Выодим таблицу

In [ ]:
df.show()

+---+------+
|age|  name|
+---+------+
| 29| Alice|
| 22|   Bob|
| 30|George|
+---+------+



Третий способ.

Этот способ просто преобразовыввает RDD в DataFrame.


In [23]:
# Создание RDD из списка словарей
rdd = spark.sparkContext.parallelize(data)

# Преобразование RDD в DataFrame с использованием имен столбцов
df = rdd.toDF(['name', 'age'])

# Вывод схемы DataFrame
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



In [6]:
df.show()

+-----+---+
| name|age|
+-----+---+
|Alice| 25|
|Artem| 26|
| Ivan| 48|
+-----+---+



# head и take

Оба метода возращают строку n. (в данном случае 1)

In [7]:
df.head(1)
df.take(1)

[Row(name='Alice', age=25)]

# Коллонны

**columns**

Возращает список всех названий стобцов.

In [10]:
colums = df.columns
colums

['name', 'age']

**dtypes**

Возращает словарь с названиями и типов данных столбцов.

In [11]:
colums = df.dtypes
colums

[('name', 'string'), ('age', 'bigint')]

# SQL функции

**Select**

Возрацает определённые столбцы

In [12]:
df.select("name").show()

+-----+
| name|
+-----+
|Alice|
|Artem|
| Ivan|
+-----+



**selectExpr**

Добаляет новый столбец

В примере `age + 1 as age_plus_one` это код sql

In [13]:
df.selectExpr("name", "age + 1 as age_plus_one").show()

+-----+------------+
| name|age_plus_one|
+-----+------------+
|Alice|          26|
|Artem|          27|
| Ivan|          49|
+-----+------------+



# функции для фильтрации данных

**filter**

Это стандартная функция из spark

In [16]:
df.filter(df["age"] > 30).show()

+----+---+
|name|age|
+----+---+
|Ivan| 48|
+----+---+



**where**

Это функция из SQLspark

In [17]:
df.where(df["age"] > 30).show()

+----+---+
|name|age|
+----+---+
|Ivan| 48|
+----+---+



# Групировка.

In [20]:
df.groupBy("age").count().show()

+---+-----+
|age|count|
+---+-----+
| 26|    1|
| 25|    1|
| 48|    1|
+---+-----+



In [24]:
df.groupBy("age").agg(
    avg("age").alias("averange_age"),
    max("age").alias("max_age")
).show()

+---+------------+-------+
|age|averange_age|max_age|
+---+------------+-------+
| 26|        26.0|     26|
| 25|        25.0|     25|
| 48|        48.0|     48|
+---+------------+-------+



# Сортировка

есть 4 способа как сортировать.

In [28]:
df.orderBy("age").show() # По возрастанию
df.orderBy((df["age"]).desc()).show() # По убыванию
df.sort("age", ascending = False).show() # По возрастанию
df.sort("age", ascending = True).show() # По убыванию

+-----+---+
| name|age|
+-----+---+
| Ivan| 48|
|Artem| 26|
|Alice| 25|
+-----+---+



# Добавление колонки

Сначала мы пиши название потом что за колонка.

In [29]:
df.withColumn("age_plis_one", df["age"] + 1).show()

+-----+---+------------+
| name|age|age_plis_one|
+-----+---+------------+
|Alice| 25|          26|
|Artem| 26|          27|
| Ivan| 48|          49|
+-----+---+------------+



# Переменнование столбца.

n - Столбце который надо переменновать
f - Новое имя столбца

```
#withColumnRenamed("n", "f")
```



In [31]:
df.withColumnRenamed("age", "fio").show()

+-----+---+
| name|fio|
+-----+---+
|Alice| 25|
|Artem| 26|
| Ivan| 48|
+-----+---+



# Удаление столбца

In [32]:
df.drop("name").show()

+---+
|age|
+---+
| 25|
| 26|
| 48|
+---+



# Тренеровка с JOIN

In [38]:
# Список словарей с данными
data1 = [
    {"id": 1, "name": "Alice"},
    {"id": 2, "name": "Bob"},
    {"id": 3, "name": "George"}
]

data2 = [
    {"id": 1, "salary": 50000},
    {"id": 2, "salary": 40000},
    {"id": 4, "salary": 60000}
]

# Создание DataFrame из списка словарей
# Spark автоматически выведет схему DataFrame
df1 = spark.createDataFrame(data1)
df2 = spark.createDataFrame(data2)

df_join = df1.join(df2, "id", "inner")

df_join.show()

+---+-----+------+
| id| name|salary|
+---+-----+------+
|  1|Alice| 50000|
|  2|  Bob| 40000|
+---+-----+------+

